# 使用MXNET训练神经网络来识别MNIST手写集

Apache MXNet 示例演示了对作为 Amazon SageMaker 高级 Python 库的一部分提供的 Amazon SageMaker sagemaker.mxnet.MXNet 估算器类的使用。它提供了 fit 方法和 deploy 方法，前者用于 Amazon SageMaker 中的模型训练，后者用于在 Amazon SageMaker 中部署生成的模型。在本练习中，将使用 Apache MXNet构建一个神经网络分类器。然后，使用 MNIST 数据库数据集 (Amazon SageMaker 在 S3 存储桶中提供) 来训练模型。

In [1]:
#初始化变量,提供包S3存储桶的名称。get_execution_role 函数会检索在创建笔记本实例时创建的 IAM 角色。
from sagemaker import get_execution_role

#Bucket location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://your-bucket-name/customcode/mxnet'

#Bucket location where results of model training are saved.
model_artifacts_location = 's3://your-bucket-name/artifacts'

#IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
#We can use the Amazon SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

高级 Python 库提供了 MXNet 类，它包含两种方法：fit (用于训练模型) 和 deploy (用于部署模型)。
entry_point – 示例仅使用一个源文件 (mnist.py)，已经在笔记本实例上提供此源文件。如果自定义代码包含在一个文件中，则仅指定 entry_point 参数；如果训练代码由多个文件组成，则还要添加 source_dir 参数。
注意
仅指定自定义代码的源。sagemaker.mxnet.MXNet 对象确定要用于模型训练的 Docker 映像。
role – IAM 在代表执行任务 时代入的 Amazon SageMaker 角色。
code_location – 希望 fit 方法 (下一步中) 将自定义 Apache MXNet 代码的 tar 存档上传到的 S3 位置。
output_path – 标识将模型训练结果 (模型构件) 保存到的 S3 位置。
train_instance_count 和 train_instance_type – 指定要用于模型训练的实例的数目和类型。
还可以通过指定 local 作为 train_instance_type 的值，并指定 1 作为 train_instance_count 的值，在本地计算机上训练模型。有关本地模式的更多信息，请参阅 Amazon SageMaker Python 开发工具包 中的 https://github.com/aws/sagemaker-python-sdk#local-mode。
Hyperparameters – 任何指定来影响模型最终质量的超参数。自定义训练代码将使用这些参数。

In [2]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.p3.2xlarge',
                        hyperparameters={'learning_rate': 0.1})

使用fit方法训练模型

In [3]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-09-04-06-48-02-016


........................
2018-09-04 06:51:45,659 INFO - root - running container entrypoint
2018-09-04 06:51:45,659 INFO - root - starting train task
2018-09-04 06:51:45,679 INFO - container_support.training - Training starting
2018-09-04 06:51:47,280 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 1, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'hosts': [u'algo-1'], u'network_interface_name': u'ethwe', u'current_host': u'algo-1'}, 'user_script_name': u'mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/o

2018-09-04 06:52:01,601 INFO - root - Epoch[8] Batch [300]#011Speed: 131736.86 samples/sec#011accuracy=0.972700
2018-09-04 06:52:01,698 INFO - root - Epoch[8] Batch [400]#011Speed: 103508.88 samples/sec#011accuracy=0.975600
2018-09-04 06:52:01,776 INFO - root - Epoch[8] Batch [500]#011Speed: 127008.52 samples/sec#011accuracy=0.975700
2018-09-04 06:52:01,862 INFO - root - Epoch[8] Train-accuracy=0.973131
2018-09-04 06:52:01,862 INFO - root - Epoch[8] Time cost=0.516
2018-09-04 06:52:01,923 INFO - root - Epoch[8] Validation-accuracy=0.964000
2018-09-04 06:52:02,020 INFO - root - Epoch[9] Batch [100]#011Speed: 103963.26 samples/sec#011accuracy=0.974851
2018-09-04 06:52:02,098 INFO - root - Epoch[9] Batch [200]#011Speed: 128829.56 samples/sec#011accuracy=0.975900
2018-09-04 06:52:02,178 INFO - root - Epoch[9] Batch [300]#011Speed: 125439.00 samples/sec#011accuracy=0.976000
2018-09-04 06:52:02,261 INFO - root - Epoch[9] Batch [400]#011Speed: 121209.00 samples/sec#011accuracy=0.979300
2018-0


Billable seconds: 157
CPU times: user 492 ms, sys: 44 ms, total: 536 ms
Wall time: 4min 42s


使用 deploy 方法部署模型

In [4]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-09-04-06-48-02-016
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-09-04-06-48-02-016


--------------------------------------------------------------!CPU times: user 280 ms, sys: 8 ms, total: 288 ms
Wall time: 5min 13s


提供了一个 HTML 画布，可用来使用鼠标绘制一个数字。测试代码将此图像发送到模型以进行推理。

In [5]:
from IPython.display import HTML
HTML(open("/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/input.html").read())

运行 predict 方法以从模型获取推理。
Raw prediction result (原始预测结果) 是模型作为推理返回的 10 个概率值的列表，对应于数字 0 到 9。在这些值中，输入数字为 7，它基于最高概率值 (0.7383657097816467)。
按顺序列出这些值，每个数字 (0 到 9) 对应一个值。模型已向它添加标签并返回 Labeled predictions (标记的预测)。
根据最高概率，我们的代码返回了 Most likely answer (最有可能的答案) (数字 7)。

In [6]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Raw prediction result:
[[1.9354635636335993e-19, 9.862841210406259e-08, 3.8618244713184424e-10, 6.26692853984423e-05, 1.6878833400402193e-12, 5.364214656336161e-16, 1.371284484610375e-23, 0.9999217987060547, 1.411845840237902e-08, 1.536494710308034e-05]]
Labeled predictions: 
[(0, 1.9354635636335993e-19), (1, 9.862841210406259e-08), (2, 3.8618244713184424e-10), (3, 6.26692853984423e-05), (4, 1.6878833400402193e-12), (5, 5.364214656336161e-16), (6, 1.371284484610375e-23), (7, 0.9999217987060547), (8, 1.411845840237902e-08), (9, 1.536494710308034e-05)]
Most likely answer: (7, 0.9999217987060547)
